In [163]:
import json
from coincurve import PrivateKey, PublicKey
from hashlib import sha256

def hash_to_curve(x_bytes):
    # Hash the secret using SHA-256
    hash_value = sha256(x_bytes).digest()
    # Create a public key Y from the hashed secret
    Y = PublicKey.from_secret(hash_value)
    return Y

def subtract_points(pt1: PublicKey, pt2:  PublicKey):
    p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
    x2, y2 = pt2.point()
    neg_pt2 = PublicKey.from_point(x2, (p - y2) % p)
    difference = PublicKey.combine_keys([pt1, neg_pt2])
    return difference


In [164]:
l1 = "/usr/local/bin/lightning-cli --lightning-dir=/tmp/l1"
l2 = "/usr/local/bin/lightning-cli --lightning-dir=/tmp/l2"

# mint 7 ecash tokens in exchange for 7 sats
mint_amount = 7

# melt 7 tokens for 7 sats
melt_amount = 7

# Define Keyset

In [194]:
pubkeys = !$l1 cashu-get-keys | jq -r .keysets[0].keys
pubkeys = json.loads(''.join(pubkeys))
pubkeys

{'1': '02b53a33e8b70644d83e5700b2dfa45b7fd7bb1ed597a6f4f2408e79d2ed316cef',
 '2': '034c307cfa8736158e2bb0435153f805e7d6d6ec95249eee32f31effe4fef87e24',
 '4': '02837b0c27e0e1c00934925031e10c41e7a2b926eb1b6e8d5846024b64b16964cc',
 '8': '0232eaf6f41725360b28ba5242243616099dce2f17aaf5fa7241bb4015d6461495',
 '16': '031b06e936ace0e0e449a5eeabcb0dc39a11b72650142cc653d008dce482004178',
 '32': '03053d1a6e05e1e12537129dad7c983cc1845279f3051df7d37956cdb09d633f29',
 '64': '02f81334344f31fe0b919bd9fa40e4a5b0b4e4d4dbd017cdf6dbe66d1fbfae1975',
 '128': '02ee1a9c0b7018150ff71ce3f2207011f034a6a5a81d40f2066105254993b4b305'}

In [195]:
privkeys = !$l1 cashu-dev-get-privkeys
privkeys = json.loads(''.join(privkeys))
privkeys

{'1': '78196135e2121c6750e0d3ed7669cc43a83107d79dbfbb253c069292ec83c59a',
 '2': '62d469a41ab8e82cb95dce66a8562f8a241d9f8c1c9dbaac41829d46d1856bb7',
 '4': '3375381bb41a9cb1b1a47d78d37e4a703332af50ef4d1661a8892ba3f0e42f8c',
 '8': '5c47dc806dac67518dd8355dc1c2b29daf73a88195913b5b1b8e5bf319d40ae7',
 '16': '2f8cae4671fadfec38df2d8afa2b57b5303c5035488e23d373b2bb96139b9992',
 '32': '226d7f6783a73927ece9009124589aaf107ea0edafb9f4c7c5358301b54e7888',
 '64': '52e4b9328d26a927e0473e806f52fa058027c649f9bfee45f8ed13c585892fcd',
 '128': '2207ce69cd468de1790ad1c8a5c9516ac6f6068067bf00b1e4545e134bb70fb8'}

# Minting and Melting Cashu Tokens

The following describes the path from:

    1. lightning --> ecash (**minting**)
    2. ecash --> lightning (**melting**)

## Minting
A wallet pays a lightning invoice in exchange for signatures from a mint on blinded messages.

### 1. Wallet requests a quote to *mint*

In [286]:
mint_quote = !$l1 -k cashu-quote-mint amount=$mint_amount unit="sat" | jq -r 
mint_quote = json.loads(''.join(mint_quote))

# quote is the unique id for this exchange
mint_id = mint_quote["quote"]
# `request` is a lightning invoice
mint_invoice = mint_quote["request"]

print("quoteId:", mint_id)
print("bolt11: ", mint_invoice)

quoteId: a2d0d00a058e8861
bolt11:  lnbcrt70n1pjeuvemsp559t02vpswnjdpgx5a9j59zaf5gldr63wka5rp2yt67t2k0c040sqpp5243q8zpykr7qf22lzrds437ft2fnkfghynsfaad0u42c6dtctfqqdqsg9hzq6twwehkjcm9xqyjw5qcqp29qx3qysgq3xd5vqspqewanl573lsrehwelw2ch8zwdj52zpkxruzn9rj3zxjj30algdfq6ye9h7svcjdrcadtn92yq0cc6dw75v5umm4hr920v8gp3kf388


### 2. Wallet pays invoice

In [288]:
!$l2 pay "$mint_invoice" | jq -r .status

complete


### 3. Check mint status

In [289]:
!$l1 cashu-check-mint "$mint_id"

{
   "quote": "a2d0d00a058e8861",
   "request": "lnbcrt70n1pjeuvemsp559t02vpswnjdpgx5a9j59zaf5gldr63wka5rp2yt67t2k0c040sqpp5243q8zpykr7qf22lzrds437ft2fnkfghynsfaad0u42c6dtctfqqdqsg9hzq6twwehkjcm9xqyjw5qcqp29qx3qysgq3xd5vqspqewanl573lsrehwelw2ch8zwdj52zpkxruzn9rj3zxjj30algdfq6ye9h7svcjdrcadtn92yq0cc6dw75v5umm4hr920v8gp3kf388",
   "paid": true,
   "expiry": 1705471419
}


### 4. Wallet requests tokens from mint
- wallet must first generate blinded messages from tokens they want

In [199]:
def generate_blinded_messages(secrets, amounts):
    assert len(secrets) == len(amounts)
    BlindedMessages =  []
    rs = []
    for s, amount in zip(secrets, amounts):
        # r is a random blinding factor
        r = PrivateKey()
        R = r.public_key
        #Y = hash_to_curve(x)
        Y = hash_to_curve(s.encode())
        # B_ = Y + rG
        B_ = PublicKey.combine_keys([Y, R]).format().hex()
        BlindedMessages.append({"amount": amount, "B_": B_})
        rs.append(r.secret)
    return BlindedMessages, rs

In [200]:
secrets = ['1', '2', '4']
amounts = [1, 2, 4]
b_messages, rs = generate_blinded_messages(secrets, amounts)

json.dumps(b_messages)

'[{"amount": 1, "B_": "026c623866ee41bb02c0035f56d4cae8c53258456ccb87b4af0ffdc7d3f9d50ad7"}, {"amount": 2, "B_": "03589779f4e561d4d515e3517a041b541e49816d2ab5a67e28a38978bb16f3bd83"}, {"amount": 4, "B_": "028c3be6fcae07b2c752ebd5d8c7f7b0ac2571e325381f0ed951db8811813ac5bb"}]'

#### 4b. wallet sends blinded msgs for  blinded sigs

In [206]:
blinded_sigs = ! /usr/local/bin/lightning-cli --lightning-dir=/tmp/l1 -k cashu-mint quote=e59c370eba4cd7d1 blinded_messages='[{"amount": 1, "B_": "026c623866ee41bb02c0035f56d4cae8c53258456ccb87b4af0ffdc7d3f9d50ad7"}, {"amount": 2, "B_": "03589779f4e561d4d515e3517a041b541e49816d2ab5a67e28a38978bb16f3bd83"}, {"amount": 4, "B_": "028c3be6fcae07b2c752ebd5d8c7f7b0ac2571e325381f0ed951db8811813ac5bb"}]'
blinded_sigs = json.loads(''.join(blinded_sigs))
blinded_sigs

[{'amount': 1,
  'id': '00b2f181c83b11aa',
  'C_': '039604586168ae53b3ed8e970357ddfb7f6b2d61f21deda521a1035a0b7ce2122a'},
 {'amount': 2,
  'id': '00b2f181c83b11aa',
  'C_': '03d54cf69a08531c0810ecfd7e528622b655c9b7827d224ca2fc593a70b940bc6f'},
 {'amount': 4,
  'id': '00b2f181c83b11aa',
  'C_': '0251cc7b574f54271e28f60c11152aaa4bb95b3226330017d4142ff8cd65c6ad2f'}]

## Melting

### 5. Wallet generates and invoice for mint to pay
- in this case `l2` will be the wallet

In [273]:
melt_invoice = ! /usr/local/bin/lightning-cli --lightning-dir=/tmp/l2 invoice 7000 $RANDOM description  | jq -r .'bolt11'
melt_invoice = melt_invoice[0]
melt_invoice

'lnbcrt70n1pjeuvtpsp55pp9hwwk4wkwcdng379u939a8c70jtp4k73kudmaj6q6q8gh5xgqpp54rm33j6a88szxj6x683s98v9rdutkrv2q9wxdm0nlfqt93zjnprsdqjv3jhxcmjd9c8g6t0dcxqyjw5qcqp29qx3qysgqy4a88tcerettxy5rnh6r87qlclfrd7r8yd5uxkqae52wf79c0adky7wkxkuekz5a8ux8m6efctme5mht4kw6yrp4jkv5kfequ5lyepcqmsfkr7'

### 6. Wallet requests a quote to *melt*

In [274]:
melt_quote = !$l1 -k cashu-quote-melt req=$melt_invoice unit="sat" | jq -r .'quote'
melt_quote = melt_quote[0]
melt_quote

'3fd66d28ce34d954'

### 7. Wallet sends tokens for the mint to melt

In [280]:
def verify_token(C, secret_bytes, k: str):
    # k*hash_to_curve(x) == C
    Y = hash_to_curve(secret_bytes)
    kY = Y.multiply(bytes.fromhex(k))
    return kY.format().hex() == C

def construct_token(C_: PublicKey, K: bytes, r: str):
    rK = PublicKey(K).multiply(r)
    # C = C_ - rK
    C = subtract_points(C_, rK)
    return C.format().hex()

def construct_inputs(blinded_sigs, rs, secrets):
    inputs = []
    for output, r, s in zip(blinded_sigs, rs, secrets):
        amount = output["amount"]
        # K is the public key for this token value
        K = bytes.fromhex(pubkeys.get(str(amount)))
        # C_ is blinded signature
        C_ = PublicKey(bytes.fromhex(output["C_"]))
        # C is unblinded signature
        C = construct_token(C_, K, r)
        assert verify_token(C, s.encode(), privkeys.get(str(amount)))
        inputs.append({
            "amount": amount,
            "C": C,
            "id": output["id"],
            "secret": s
        })
    return json.dumps(inputs)
    


In [281]:
outputs = blinded_sigs
construct_inputs(outputs, rs, secrets)


'[{"amount": 1, "C": "02114511fb00338ea95280db720c63ae119c68c03941df3a61223ed95e76ff9574", "id": "00b2f181c83b11aa", "secret": "1"}, {"amount": 2, "C": "02853da76d8dedbbcece373e312007cc3bb92711380c03a0ca96c56f427a0a50e0", "id": "00b2f181c83b11aa", "secret": "2"}, {"amount": 4, "C": "02e78ba0328f6947671a7964bbc1a433231f36c86090f8203ef9a6ea81869e8b6b", "id": "00b2f181c83b11aa", "secret": "4"}]'

In [277]:
!/usr/local/bin/lightning-cli --lightning-dir=/tmp/l1 -k cashu-melt quote=3fd66d28ce34d954 inputs='[{"amount": 1, "C": "02114511fb00338ea95280db720c63ae119c68c03941df3a61223ed95e76ff9574", "id": "00b2f181c83b11aa", "secret": "1"}, {"amount": 2, "C": "02853da76d8dedbbcece373e312007cc3bb92711380c03a0ca96c56f427a0a50e0", "id": "00b2f181c83b11aa", "secret": "2"}, {"amount": 4, "C": "02e78ba0328f6947671a7964bbc1a433231f36c86090f8203ef9a6ea81869e8b6b", "id": "00b2f181c83b11aa", "secret": "4"}]'

{
   "paid": true,
   "preimage": "a0f3c06d1533df0cf8d849cbff228cb95b85205851731640d535d375fbda03b7"
}


### 6. Check melt status

In [279]:
!$l1 cashu-check-melt 3fd66d28ce34d954

{
   "quote": "3fd66d28ce34d954",
   "amount": "7msat",
   "fee_reserve": 0,
   "paid": true,
   "expiry": 0
}
